In [28]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

from google.colab import drive
drive.mount('/content/drive')


Authenticated
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
!pip install gspread --upgrade # Uninstall gspread-3.0.1 with 3.7.0
import gspread
from gspread.models import Cell, Spreadsheet
from oauth2client.client import GoogleCredentials as GC
import random
import string
from datetime import date, time, datetime, timedelta
gs_ = gspread.authorize(GC.get_application_default())

Requirement already up-to-date: gspread in /usr/local/lib/python3.7/dist-packages (3.7.0)


In [30]:
def getGworkbook(ref, client):
  """
  ref: is a gsheet url or key
  """
  try:
    return client.open_by_url(ref)
  except:
    return client.open_by_key(ref)

def genFileName(N):
  # import random
  # import string
  return ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(N))

def genWeekEndingsList(initDate, n=52, ascending=True):
  ll = list()
  ll.append(initDate)
  if ascending:
    inc_=7
  else:
    inc_=-7
  for i in range(n):
    dateDiff = timedelta(days=inc_*(i+1))
    date2Append = initDate+dateDiff
    ll.append(date2Append)
  return ll

# def addSheetsToSpreadsheet(gs_spreadsheet,newSheetNames,source_sheet_id=0):
#   for i in range(1,len(newSheetNames)):
#     gs_spreadsheet.duplicate_sheet(source_sheet_id=source_sheet_id, insert_sheet_index=i, new_sheet_id=i, new_sheet_name=newSheetNames[i-1])


def addSheetToSpreadsheet(gs_spreadsheet,source_sheet_id, insert_sheet_index, new_sheet_id, new_sheet_name):
  gs_spreadsheet.duplicate_sheet(source_sheet_id=source_sheet_id, insert_sheet_index=insert_sheet_index, new_sheet_id=new_sheet_id, new_sheet_name=new_sheet_name)

def getStoreListGsheet(gSheetID, gspreadAuth):
  # get store list gsheet
  import numpy as np
  storeList_worksheet = getGworkbook(gSheetID, gspreadAuth).sheet1
  storeList_array = np.array(storeList_worksheet.get_all_values())[1:]

  store_nums = storeList_array.T[0]
  store_gsheet_names = storeList_array.T[1]
  store_name_in_gsheet = storeList_array.T[2]

  # clean access names:
  access_name_separator = ';'
  cleaned_perm_emails = [x[3].split(access_name_separator) for x in storeList_array]

  ll = []
  for i in range(len(cleaned_perm_emails)):
    ll.append( [s.strip() for s in cleaned_perm_emails[i]] )
  cleaned_perm_emails = ll

  return store_nums, store_gsheet_names, cleaned_perm_emails


In [35]:
## inputs
templateGsheetID = '1ad6wWzYoUW_jmjkImCAtbePsdrospUAnMlqC9ZDlx2Q' # this is the ID of the template gsheet
initial_date = date(year=2022, month=6, day=28)

storeListGsheetID = '1j2oKTNI8C5k7Do4ZTyZa-Zsj9agUMe1PXuSM5tkHp-8' # this is the store list gsheet ID - with store numbers, store names and permission emails

we_dates = genWeekEndingsList(initial_date,n=52, ascending=False)
we_dates_str = [str(x) for x in we_dates]


In [36]:
# get template gsheet
template_wb = getGworkbook(templateGsheetID, gs_)

## get store list gsheet
store_nums, store_gsheet_names, cleaned_perm_emails = getStoreListGsheet(gSheetID=storeListGsheetID, gspreadAuth=gs_)

In [38]:
# add a new sheet 1 for each week
# and update + protect range

for sn in range(len(store_nums)):
  c_store_nums, c_store_gsheet_names, c_cleaned_perm_emails = store_nums[sn], store_gsheet_names[sn], cleaned_perm_emails[sn]
  
  # create new sheet
  client = gspread.authorize(GC.get_application_default())
  fName = c_store_gsheet_names
  ## create new file as copy of template
  newGsheet = client.copy(templateGsheetID, title=fName, copy_permissions=True)

  for i in range(1,len(we_dates_str)):
    i_we_dates_str = we_dates_str[i-1]
    addSheetToSpreadsheet(gs_spreadsheet=newGsheet,source_sheet_id=0, insert_sheet_index=i, new_sheet_id=i, new_sheet_name=i_we_dates_str)
    worksheet = newGsheet.worksheet(i_we_dates_str)

    # update date in sheet
    worksheet.update_acell('F2', i_we_dates_str)
    # update store number in sheet
    worksheet.update_acell('B3', c_store_nums)



KeyboardInterrupt: ignored